In [1]:
%%html
<style>@import url('../style.css')</style><script>IPython.OutputArea.prototype._should_scroll = function(){return false}</script>

In [2]:
import cPickle

def pretty_confusion(confusion_matrix, df, mode='recall', css_classes=['diagonal', 'cell_right'], raw=False):
    if mode == 'recall':
        confusion_matrix = confusion_matrix * 1000 / np.sum(confusion_matrix, axis=1) / 10.0
        confusion_matrix = np.vectorize(lambda x: '{0}%'.format(x))(confusion_matrix)

    show_headers = False if 'draggable' in css_classes else True

    categories = sorted(df['category'].unique())
    labels = map(lambda c: df[df['category'] == c]['category_name'][0:1].values[0], categories)
    confusion_matrix = pd.DataFrame(confusion_matrix, index=labels, columns=labels)

    if raw:
        return confusion_matrix    
    else:
        return IPython.display.HTML(confusion_matrix.to_html(classes=css_classes, header=show_headers))
        
def evaluate_model(dataset, model, fold, filename):
    import sklearn as sk

    print 'Testing on fold {}'.format(fold)
    accuracy = dataset.test_model(model)
    aggregated = dataset.test.groupby('filename', sort=False)

    if NET_MODE == 'majority':        
        aggregated = aggregated[['category', 'category_name', 'prediction']].agg(lambda x: x.value_counts().index[0])
    else:
        cols = map(lambda x: 'prediction' + str(x), range(np.shape(dataset.train.y)[1]))
        aggregated = aggregated[['category'] + cols].agg(np.mean)
        aggregated.loc[:, 'prediction'] = np.argmax(aggregated[cols].as_matrix(), axis=1)
        
    conf = sk.metrics.confusion_matrix(aggregated['category'], aggregated['prediction'])

    with open(filename, 'w') as f:
        np.savetxt(f, conf)

    return conf, aggregated        